<a href="https://colab.research.google.com/github/Yundo37/automation-portfolio-YB/blob/main/Selenium_Pytest_%EC%9E%90%EB%8F%99%ED%99%94_%ED%85%8C%EC%8A%A4%ED%8A%B8_%ED%94%84%EB%A0%88%EC%9E%84%EC%9B%8C%ED%81%AC_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Selenium Pytest 자동화 테스트 프레임워크**

💡채용 담당자님께: 이 노트북은 클릭 한 번으로 실행되며, 별도의 설정 없이 테스트 결과를 확인할 수 있습니다.

실행 순서

1. 런타임 > 모두 실행 클릭
2. 왼쪽 파일 패널에서 프로젝트 구조 확인
3. 테스트 실행 과정 실시간 확인
4. HTML 리포트 및 스크린샷 결과 확인

In [1]:
# 필요한 패키지 설치
!pip install selenium pytest pytest-html webdriver-manager --quiet

# Chrome 및 ChromeDriver 설치
!apt-get update --quiet
!apt-get install -y chromium-browser --quiet

print("환경 설정 완료!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 30.8 MB/s eta 0:00:00
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,933 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports I

In [4]:
# @title
import os
import shutil

# 기존 디렉토리 제거 후 새로 생성
if os.path.exists('/content/selenium-pytest-framework'):
    shutil.rmtree('/content/selenium-pytest-framework')

print("프로젝트 디렉토리 구조 생성 중...")

# 디렉토리 구조 생성
directories = [
    'selenium-pytest-framework/tests',
    'selenium-pytest-framework/pageObjects',
    'selenium-pytest-framework/utilities',
    'selenium-pytest-framework/testData',
    'selenium-pytest-framework/reports'
]

for dir_path in directories:
    os.makedirs(f'/content/{dir_path}', exist_ok=True)
    # __init__.py 파일 생성
    if any(folder in dir_path for folder in ['tests', 'pageObjects', 'utilities', 'testData']):
        with open(f'/content/{dir_path}/__init__.py', 'w') as f:
            f.write("# Package initialization file\n")

print("프로젝트 구조 생성 완료!")
print("왼쪽 파일 패널에서 생성된 폴더 구조를 확인하세요.")

프로젝트 디렉토리 구조 생성 중...
프로젝트 구조 생성 완료!
왼쪽 파일 패널에서 생성된 폴더 구조를 확인하세요.


In [3]:
# @title
print("BaseClass.py (공통 유틸리티) 생성 중...")

# BaseClass.py - 공통 유틸리티 (원본 코드 그대로)
base_class_code = '''import inspect
import logging

import pytest
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.wait import WebDriverWait


@pytest.mark.usefixtures("setup")
class BaseClass:

    def getLogger(self):
        loggerName = inspect.stack()[1][3]
        logger = logging.getLogger(loggerName)
        fileHandler = logging.FileHandler('logfile.log')
        formatter = logging.Formatter("%(asctime)s :%(levelname)s : %(name)s : %(message)s")
        fileHandler.setFormatter(formatter)

        logger.addHandler(fileHandler)  # filehandler object

        logger.setLevel(logging.DEBUG)
        return logger


    def verifyLinkPresence(self, text):
        wait = WebDriverWait(self.driver, 10)
        wait.until(expected_conditions.presence_of_element_located((By.LINK_TEXT, text)))

    def selectOptionByText(self,element,text):
        sel = Select(element)
        sel.select_by_visible_text(text)

    def selectOptionByIndex(self, element, index):
        sel = Select(element)
        sel.select_by_index(index)
'''

with open('/content/selenium-pytest-framework/utilities/BaseClass.py', 'w') as f:
    f.write(base_class_code)

print("utilities/BaseClass.py 생성 완료!")

BaseClass.py (공통 유틸리티) 생성 중...
utilities/BaseClass.py 생성 완료!


In [5]:
# @title
print("conftest.py (Pytest 설정) 생성 중...")

# conftest.py - 헤드리스 브라우저 설정
conftest_code = '''import pytest
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

driver = None

def pytest_addoption(parser):
    parser.addoption("--browser_name", action="store", default="chrome")

@pytest.fixture(scope="class")
def setup(request):
    global driver
    print("브라우저 설정 시작...")

    # Chrome 옵션 설정 (Colab용 헤드리스)
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920,1080")

    # ChromeDriver 자동 관리
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)

    print("테스트 사이트 접속 중...")
    driver.get("https://rahulshettyacademy.com/angularpractice/")
    driver.implicitly_wait(5)

    request.cls.driver = driver
    print("브라우저 설정 완료!")

    yield

    print("브라우저 종료 중...")
    driver.quit()

@pytest.mark.hookwrapper
def pytest_runtest_makereport(item):
    pytest_html = item.config.pluginmanager.getplugin('html')
    outcome = yield
    report = outcome.get_result()
    extra = getattr(report, 'extra', [])

    if report.when == 'call' or report.when == "setup":
        xfail = hasattr(report, 'wasxfail')
        if (report.skipped and xfail) or (report.failed and not xfail):
            file_name = f"/content/selenium-pytest-framework/reports/{report.nodeid.replace('::', '_')}.png"
            _capture_screenshot(file_name)
            print(f"실패 스크린샷 저장: {file_name}")

            if file_name:
                html = f'<div><img src="{file_name}" alt="screenshot" style="width:600px;height:400px;" onclick="window.open(this.src)" align="right"/></div>'
                extra.append(pytest_html.extras.html(html))
        report.extra = extra

def _capture_screenshot(name):
    if driver:
        driver.save_screenshot(name)
        print(f"스크린샷 캡처됨: {name}")
'''

with open('/content/selenium-pytest-framework/tests/conftest.py', 'w') as f:
    f.write(conftest_code)

print("tests/conftest.py 생성 완료!")

conftest.py (Pytest 설정) 생성 중...
tests/conftest.py 생성 완료!


In [6]:
# @title
print("Page Object 클래스들 생성 중...")

# HomePage.py (원본 코드 그대로)
homepage_code = '''from selenium.webdriver.common.by import By
from pageObjects.CheckoutPage import CheckOutPage


class HomePage:
    def __init__(self, driver):
        self.driver = driver

    shop = (By.CSS_SELECTOR, "a[href*='shop']")
    name = (By.CSS_SELECTOR, "input[name='name']")
    email = (By.NAME, "email")
    password = (By.ID, "exampleInputPassword1")
    checkbox = (By.ID, "exampleCheck1")
    genderDropdown = (By.ID, "exampleFormControlSelect1")
    employmentStatus = (By.CSS_SELECTOR, "#inlineRadio1")
    submitButton = (By.XPATH, "//input[@type='submit']")
    successText = (By.CLASS_NAME, "alert-success")
    twoWayBindingField = (By.XPATH, "(//input[@type='text'])[3]")


    def shopItems(self):
        self.driver.find_element(*HomePage.shop).click()
        checkoutpage = CheckOutPage(self.driver)
        return checkoutpage

    def getName(self):
        return self.driver.find_element(*HomePage.name)

    def getEmail(self):
        return self.driver.find_element(*HomePage.email)

    def getPassword(self):
        return self.driver.find_element(*HomePage.password)

    def selectCheckbox(self):
        return self.driver.find_element(*HomePage.checkbox)

    def getGenderDropdown(self):
        return self.driver.find_element(*HomePage.genderDropdown)

    def selectEmploymentStatus(self):
        return self.driver.find_element(*HomePage.employmentStatus)

    def submitForm(self):
        return self.driver.find_element(*HomePage.submitButton)

    def getSuccessMessage(self):
        return self.driver.find_element(*HomePage.successText).text

    def getTwoWayBindingField(self):
        return self.driver.find_element(*HomePage.twoWayBindingField)
'''

with open('/content/selenium-pytest-framework/pageObjects/HomePage.py', 'w') as f:
    f.write(homepage_code)

# CheckoutPage.py (원본 코드 그대로)
checkout_code = '''from selenium.webdriver.common.by import By

from pageObjects.ConfirmPage import ConfirmPage

class CheckOutPage:
    def __init__(self, driver):
        self.driver = driver

    productTitle = (By.XPATH, "//div[@class='card h-100']")
    productFooter = (By.XPATH, "div/button")
    checkOut = (By.XPATH, "//button[@class='btn btn-success']")

    def getProductTitles(self):
        return self.driver.find_elements(*CheckOutPage.productTitle)

    def getProductFooter(self, product):
        return product.find_element(*CheckOutPage.productFooter)

    def CheckOutItems(self):
        self.driver.find_element(*CheckOutPage.checkOut).click()
        confirmPage = ConfirmPage(self.driver)
        return confirmPage
'''

with open('/content/selenium-pytest-framework/pageObjects/CheckoutPage.py', 'w') as f:
    f.write(checkout_code)

# ConfirmPage.py (원본 코드 그대로)
confirm_code = '''from selenium.webdriver.common.by import By


class ConfirmPage:

    def __init__(self, driver):
        self.driver = driver

    countryInput = (By.ID, "country")
    countrySelect = (By.LINK_TEXT, "India")
    checkbox = (By.XPATH, "//div[@class='checkbox checkbox-primary']")
    submitButton = (By.CSS_SELECTOR, "[type='submit']")
    successMsg = (By.CLASS_NAME, "alert-success")

    def enterCountry(self, name):
        return self.driver.find_element(*ConfirmPage.countryInput).send_keys(name)

    def selectCountry(self):
        return self.driver.find_element(*ConfirmPage.countrySelect)

    def checkTerms(self):
        return self.driver.find_element(*ConfirmPage.checkbox)

    def submitOrder(self):
        return self.driver.find_element(*ConfirmPage.submitButton)

    def getSuccessMessage(self):
        return self.driver.find_element(*ConfirmPage.successMsg).text
'''

with open('/content/selenium-pytest-framework/pageObjects/ConfirmPage.py', 'w') as f:
    f.write(confirm_code)

print("pageObjects/ 폴더의 모든 클래스 생성 완료!")

Page Object 클래스들 생성 중...
pageObjects/ 폴더의 모든 클래스 생성 완료!


In [7]:
# @title
print("테스트 데이터 파일 생성 중...")

# HomePageData.py (원본 코드 그대로)
test_data_code = '''class HomePageData:

    test_Homepage_data = [{"username":"YB", "email":"nohoho37@gmail.com", "gender":"Male"}, {"username":"OB", "email":"porore37@naver.com", "gender":"Female"}]'''

with open('/content/selenium-pytest-framework/testData/HomePageData.py', 'w') as f:
    f.write(test_data_code)

print("testData/HomePageData.py 생성 완료!")

테스트 데이터 파일 생성 중...
testData/HomePageData.py 생성 완료!


In [ ]:
print("E2E 테스트 파일 생성 중...")

# test_e2e.py (원본 코드에 프린트, 로깅 추가)
e2e_test_code = '''import time
from selenium.webdriver.common.by import By
from pageObjects.HomePage import HomePage
from utilities.BaseClass import BaseClass


class TestOne(BaseClass):

    def test_e2e(self):
        print("=" * 60)
        print("E2E 테스트 시작!")
        print("=" * 60)

        log = self.getLogger()
        homePage = HomePage(self.driver)

        print("쇼핑 페이지로 이동 중...")
        checkoutpage = homePage.shopItems()

        log.info("getting all the product title")
        print("상품 목록 조회 중...")
        products = checkoutpage.getProductTitles()

        for product in products:
            productName = product.find_element(By.XPATH, "div/h4/a").text
            print(f"상품 확인: {productName}")
            log.info(productName)
            if productName == "Blackberry":
                print(f"'{productName}' 상품을 장바구니에 추가!")
                checkoutpage.getProductFooter(product).click()
                break

        print("체크아웃 페이지로 이동...")
        self.driver.find_element(By.CSS_SELECTOR, "a[class*=btn-primary]").click()

        confirmPage = checkoutpage.CheckOutItems()
        log.info("Entering country name is ind")
        print("국가 입력: ind")
        confirmPage.enterCountry("ind")
        self.verifyLinkPresence("India")

        print("India 국가 선택")
        confirmPage.selectCountry().click()

        print("약관 동의")
        confirmPage.checkTerms().click()

        print("주문 제출")
        confirmPage.submitOrder().click()

        successText = confirmPage.getSuccessMessage()
        print(f"주문 결과: {successText}")
        log.info("Text received from application is" + successText)
        assert "Success! Thank you!" in successText

        print("=" * 60)
        print("E2E 테스트 성공적으로 완료!")
        print("=" * 60)

        time.sleep(3)
'''

with open('/content/selenium-pytest-framework/tests/test_e2e.py', 'w') as f:
    f.write(e2e_test_code)

print("tests/test_e2e.py 생성 완료!")

# E2E 테스트 실행
import os
os.chdir('/content/selenium-pytest-framework')

print("\nE2E 테스트 실행 시작!")
print("=" * 50)

!pytest tests/test_e2e.py -v -s --html=reports/e2e_report.html --self-contained-html

print("E2E 테스트 완료!")

In [ ]:
print("홈페이지 폼 테스트 파일 생성 중...")

# test_HomePage.py (원본 코드에 의도적 실패 추가)
homepage_test_code = '''import pytest

from pageObjects.HomePage import HomePage
from testData.HomePageData import HomePageData
from utilities.BaseClass import BaseClass


class TestHomePage(BaseClass):

    def test_formSubmission(self, getData):
        print("=" * 60)
        print(f"홈페이지 폼 테스트 시작 - 사용자: {getData['username']}")
        print("=" * 60)

        log = self.getLogger()
        homepage = HomePage(self.driver)

        log.info("user name is" + getData["username"])
        print(f"사용자명 입력: {getData['username']}")
        homepage.getName().send_keys(getData["username"])

        print(f"이메일 입력: {getData['email']}")
        homepage.getEmail().send_keys(getData["email"])

        print("비밀번호 입력")
        homepage.getPassword().send_keys("123456")

        print("체크박스 선택")
        homepage.selectCheckbox().click()

        print("고용 상태 선택")
        homepage.selectEmploymentStatus().click()

        print(f"성별 선택: {getData['gender']}")
        self.selectOptionByText(homepage.getGenderDropdown(),getData["gender"])
        self.selectOptionByIndex(homepage.getGenderDropdown(), 0)

        print("폼 제출")
        homepage.submitForm().click()

        message = homepage.getSuccessMessage()
        print(f"폼 제출 결과: {message}")

        # 첫 번째 사용자에게는 의도적으로 잘못된 검증 추가 (실패 시나리오)
        if getData["username"] == "YB":
            print("의도적 실패 시나리오 실행 (스크린샷 캡처용)")
            assert "WRONG_MESSAGE" in message, "이것은 의도적인 실패입니다 - HTML 리포트 및 스크린샷 확인용"
        else:
            assert "Success" in message

        self.driver.refresh()

        print("=" * 60)
        print(f"{getData['username']} 폼 테스트 완료!")
        print("=" * 60)


    @pytest.fixture(params=HomePageData.test_Homepage_data)
    def getData(self, request):
        return request.param
'''

with open('/content/selenium-pytest-framework/tests/test_HomePage.py', 'w') as f:
    f.write(homepage_test_code)

print("tests/test_HomePage.py 생성 완료!")

# 홈페이지 테스트 실행
print("\n홈페이지 폼 테스트 실행 시작!")
print("=" * 50)

!pytest tests/test_HomePage.py -v -s --html=reports/homepage_report.html --self-contained-html

print("홈페이지 폼 테스트 완료!")

In [ ]:
import os
from IPython.display import HTML, display
from google.colab import files

print("HTML 리포트 및 스크린샷 확인")
print("=" * 50)

# 생성된 파일들 확인
reports_dir = "/content/selenium-pytest-framework/reports"
if os.path.exists(reports_dir):
    files_in_reports = os.listdir(reports_dir)

    html_files = [f for f in files_in_reports if f.endswith('.html')]
    screenshot_files = [f for f in files_in_reports if f.endswith('.png')]

    print(f"생성된 HTML 리포트 ({len(html_files)}개):")
    for html_file in html_files:
        file_path = os.path.join(reports_dir, html_file)
        file_size = os.path.getsize(file_path) / 1024
        print(f"  - {html_file} ({file_size:.2f} KB)")

    if screenshot_files:
        print(f"\n캡처된 스크린샷 ({len(screenshot_files)}개):")
        for screenshot in screenshot_files:
            file_path = os.path.join(reports_dir, screenshot)
            file_size = os.path.getsize(file_path) / 1024
            print(f"  - {screenshot} ({file_size:.2f} KB)")
    else:
        print("\n캡처된 스크린샷이 없습니다.")

# HTML 리포트 미리보기 (첫 번째 리포트)
if html_files:
    first_report = html_files[0]
    report_path = os.path.join(reports_dir, first_report)

    print(f"\n{first_report} 미리보기:")
    print("=" * 40)

    try:
        with open(report_path, 'r', encoding='utf-8') as f:
            html_content = f.read()

        # HTML 리포트의 요약 부분만 표시
        if "summary" in html_content.lower():
            print("HTML 리포트가 성공적으로 생성되었습니다!")
            print("리포트에는 테스트 결과, 실행 시간, 스크린샷이 포함되어 있습니다.")
        else:
            print("HTML 리포트 내용을 확인할 수 없습니다.")

    except Exception as e:
        print(f"HTML 리포트 읽기 실패: {e}")

# 파일 다운로드 준비
print(f"\n테스트 결과 다운로드:")
print("=" * 40)

# reports 폴더 전체를 압축
import shutil
shutil.make_archive('/content/test_results', 'zip', reports_dir)

print("모든 테스트 결과가 test_results.zip으로 패키징되었습니다!")
print("\n포함된 파일들:")
print("  - HTML 리포트 (테스트 결과 상세)")
print("  - PNG 스크린샷 (실패 시 화면 캡처)")

# 파일 다운로드
try:
    files.download('/content/test_results.zip')
    print("\n파일 다운로드가 시작되었습니다!")
    print("다운로드된 파일을 압축 해제하여 HTML 리포트를 브라우저에서 열어보세요!")
except Exception as e:
    print(f"\n자동 다운로드 실패: {e}")
    print("수동으로 다운로드하세요: /content/test_results.zip")

print(f"\n왼쪽 파일 패널 > selenium-pytest-framework > reports 폴더에서도 결과를 확인할 수 있습니다!")